In [33]:
import pandas as pd
import numpy as np
import os
import re
import openpyxl
from pymongo import MongoClient
import json

folder_paths = {
    'Flipkart': r'D:\E-commerce\Automation Source\Flipkart',
    'Amazon': r'D:\E-commerce\Automation Source\Amazon',
    'Meesho_Invoice': r'D:\E-commerce\Automation Source\Meesho'
}

def get_file_paths(folder, ext, pattern=None, exclude_pattern=None):
    files = [
        f for f in os.listdir(folder)
        if f.endswith(ext)
        and (pattern is None or re.search(pattern, f))
        and (exclude_pattern is None or not re.search(exclude_pattern, f))
    ]
    return [os.path.join(folder, f) for f in files]

def read_excel_files(file_paths, sheet_name):
    return pd.concat([pd.read_excel(fp, sheet_name=sheet_name) for fp in file_paths], ignore_index=True)

def read_csv_files(file_paths):
    return pd.concat([pd.read_csv(fp) for fp in file_paths], ignore_index=True)

flipkart_files = get_file_paths(folder_paths['Flipkart'], '.xlsx',exclude_pattern=None)
flipkart_data = read_excel_files(flipkart_files, sheet_name='Sales Report')

amazon_b2b_files = get_file_paths(folder_paths['Amazon'], '.csv', 'B2B',exclude_pattern=None)
amazon_b2c_files = get_file_paths(folder_paths['Amazon'], '.csv', 'B2C',exclude_pattern=None)
amazon_b2b_data = read_csv_files(amazon_b2b_files)
amazon_b2c_data = read_csv_files(amazon_b2c_files)

meesho_invoice_files = get_file_paths(folder_paths['Meesho_Invoice'], '.xlsx', 'Tax_invoice_details',exclude_pattern=None)
meesho_forward_files = get_file_paths(folder_paths['Meesho_Invoice'], '.xlsx', 'Forward',exclude_pattern=None)
meesho_reverse_files = get_file_paths(folder_paths['Meesho_Invoice'], '.xlsx', 'Reverse',exclude_pattern='Waiver')



meesho_invoice_data = read_excel_files(meesho_invoice_files, sheet_name='Invoice_Info')
meesho_forward_data = read_excel_files(meesho_forward_files, sheet_name='282410')
meesho_reverse_data = read_excel_files(meesho_reverse_files, sheet_name='282410')

def get_financial_year(date):
    year = date.year
    if date.month >= 4:
        fy_start_year = year
    else: 
        fy_start_year = year - 1
    fy_end_year = fy_start_year + 1
    return f"{fy_start_year}-{str(fy_end_year)[2:]}"

def add_financial_year(df, date_col):
    df[date_col] = pd.to_datetime(df[date_col])
    df['Financial Year'] = df[date_col].apply(lambda x: f"{x.year if x.month >= 4 else x.year - 1}-{(x.year if x.month >= 4 else x.year - 1) + 1 - 2000}")
    return df

flipkart_data = add_financial_year(flipkart_data, 'Buyer Invoice Date')
amazon_b2b_data = add_financial_year(amazon_b2b_data, 'Invoice Date')
amazon_b2c_data = add_financial_year(amazon_b2c_data, 'Invoice Date')

Flipkart_Tax_report = pd.DataFrame({
    'Marketplace': 'FLIPKART',
    'Invoice Number': flipkart_data['Buyer Invoice ID'],
    'Invoice Date': flipkart_data['Buyer Invoice Date'],
    'Party Name': 'NA',
    'GSTN': 'NA',
    'Period': pd.to_datetime(flipkart_data['Buyer Invoice Date']).dt.strftime('%b'),
    'FY': flipkart_data['Financial Year'],
    'Nature': 'B2C',
    'Transaction Type': flipkart_data['Event Sub Type'],
    'Destination': flipkart_data['Customer\'s Delivery State'],
    'HSN Code': flipkart_data['HSN Code'],
    'Qty': flipkart_data['Item Quantity'],
    'Taxable Value': flipkart_data['Taxable Value (Final Invoice Amount -Taxes)'],
    'GST Rate': (flipkart_data['IGST Rate'] + flipkart_data['CGST Rate'] + flipkart_data['SGST Rate (or UTGST as applicable)']) / 100,
    'IGST': flipkart_data['IGST Amount'],
    'CGST': flipkart_data['CGST Amount'],
    'SGST': flipkart_data['SGST Amount (Or UTGST as applicable)'],
    'Invoice Value': flipkart_data['Buyer Invoice Amount'],
    'TCS': flipkart_data['Total TCS Deducted'],
})

B2B_Amazon_Tax_Report = pd.DataFrame({
    'Marketplace': 'AMAZON',
    'Invoice Number': amazon_b2b_data['Invoice Number'],
    'Invoice Date': amazon_b2b_data['Invoice Date'],
    'Party Name': amazon_b2b_data['Buyer Name'],
    'GSTN': amazon_b2b_data['Customer Bill To Gstid'],
    'Period': pd.to_datetime(amazon_b2b_data['Invoice Date']).dt.strftime('%b'),
    'FY': amazon_b2b_data['Financial Year'],
    'Nature': 'B2B',
    'Transaction Type': amazon_b2b_data['Transaction Type'],
    'Destination': amazon_b2b_data['Ship To State'],
    'HSN Code': amazon_b2b_data['Hsn/sac'],
    'Qty': amazon_b2b_data['Quantity'],
    'Taxable Value': amazon_b2b_data['Tax Exclusive Gross'],
    'GST Rate': amazon_b2b_data['Igst Rate'] + amazon_b2b_data['Cgst Rate'] + amazon_b2b_data['Sgst Rate'] + amazon_b2b_data['Utgst Rate'],
    'IGST': amazon_b2b_data['Tax Exclusive Gross'] * amazon_b2b_data['Igst Rate'],
    'CGST': amazon_b2b_data['Tax Exclusive Gross'] * amazon_b2b_data['Cgst Rate'],
    'SGST': amazon_b2b_data['Tax Exclusive Gross'] * (amazon_b2b_data['Sgst Rate'] + amazon_b2b_data['Utgst Rate']),
    'Invoice Value': amazon_b2b_data['Invoice Amount'],
    'TCS': amazon_b2b_data['Tax Exclusive Gross'] * 0.01
})

B2C_Amazon_Tax_Report = pd.DataFrame({
    'Marketplace': 'AMAZON',
    'Invoice Number': amazon_b2c_data['Invoice Number'],
    'Invoice Date': amazon_b2c_data['Invoice Date'],
    'Party Name': 'NA',
    'GSTN': 'NA',
    'Period': pd.to_datetime(amazon_b2c_data['Invoice Date']).dt.strftime('%b'),
    'FY': amazon_b2c_data['Financial Year'],
    'Nature': 'B2C',
    'Transaction Type': amazon_b2c_data['Transaction Type'],
    'Destination': amazon_b2c_data['Ship To State'],
    'HSN Code': amazon_b2c_data['Hsn/sac'],
    'Qty': amazon_b2c_data['Quantity'],
    'Taxable Value': amazon_b2c_data['Tax Exclusive Gross'],
    'GST Rate': amazon_b2c_data['Igst Rate'] + amazon_b2c_data['Cgst Rate'] + amazon_b2c_data['Sgst Rate'] + amazon_b2c_data['Utgst Rate'],
    'IGST': amazon_b2c_data['Tax Exclusive Gross'] * amazon_b2c_data['Igst Rate'],
    'CGST': amazon_b2c_data['Tax Exclusive Gross'] * amazon_b2c_data['Cgst Rate'],
    'SGST': amazon_b2c_data['Tax Exclusive Gross'] * (amazon_b2c_data['Sgst Rate'] + amazon_b2c_data['Utgst Rate']),
    'Invoice Value': amazon_b2c_data['Invoice Amount'],
    'TCS': amazon_b2c_data['Tax Exclusive Gross'] * 0.01
})


filtered_data_1 = meesho_invoice_data[meesho_invoice_data['Type'] == 'INVOICE']
Meesho_Merged = pd.merge(meesho_forward_data,filtered_data_1,how='left',left_on='sub_order_num',right_on='Suborder No.')
Meesho_Merged
Meesho_Merged['Order Date'] = pd.to_datetime(Meesho_Merged['Order Date'])
Meesho_Merged['Financial Year'] = Meesho_Merged['Order Date'].apply(get_financial_year)
Meesho_Merged_Forward_Report = pd.DataFrame({
    'Marketplace': 'MEESHO',
    'Invoice Number': Meesho_Merged['Invoice No.'],
    'Invoice Date': Meesho_Merged['Order Date'],
    'Party Name': 'NA',
    'GSTN': 'NA',
    'Period': pd.to_datetime(Meesho_Merged['Order Date']).dt.strftime('%b'),
    'FY': Meesho_Merged['Financial Year'],
    'Nature': 'B2C',
    'Transaction Type': Meesho_Merged['order_status'],
    'Destination': Meesho_Merged['end_customer_state_new'],
    'HSN Code': Meesho_Merged['HSN'],
    'Qty': Meesho_Merged['quantity'],
    'Taxable Value': Meesho_Merged['tcs_taxable_amount'],
    'GST Rate': Meesho_Merged['gst_rate'] / 100,
    'IGST': 
    Meesho_Merged.apply(
    lambda row: row['tcs_taxable_amount'] * (row['gst_rate'] / 100) if row['end_customer_state_new'] != 'Tamil Nadu' else 0,
    axis=1),
    'CGST': 
    Meesho_Merged.apply(
    lambda row: row['tcs_taxable_amount'] * ((row['gst_rate']  / 100) / 2) if row['end_customer_state_new'] == 'Tamil Nadu' else 0,
    axis=1),
    'SGST': 
    Meesho_Merged.apply(
    lambda row: row['tcs_taxable_amount'] * ((row['gst_rate']  / 100) / 2) if row['end_customer_state_new'] == 'Tamil Nadu' else 0,
    axis=1),
    'Invoice Value': Meesho_Merged['meesho_price'],
    'TCS': Meesho_Merged['tcs_taxable_amount'] * 0.01
})

filtered_data_2 = meesho_invoice_data[meesho_invoice_data['Type'] == 'CREDIT INVOICE']
Meesho_Merged_1 = pd.merge(meesho_reverse_data,filtered_data_2,how='left',left_on='sub_order_num',right_on='Suborder No.')
Meesho_Merged_1
Meesho_Merged_1['Order Date'] = pd.to_datetime(Meesho_Merged_1['Order Date'])
Meesho_Merged_1['Financial Year'] = Meesho_Merged_1['Order Date'].apply(get_financial_year)
Meesho_Merged_1_Reverse_Report = pd.DataFrame({
    'Marketplace': 'MEESHO',
    'Invoice Number': Meesho_Merged_1['Invoice No.'],
    'Invoice Date': Meesho_Merged_1['Order Date'],
    'Party Name': 'NA',
    'GSTN': 'NA',
    'Period': pd.to_datetime(Meesho_Merged_1['Order Date']).dt.strftime('%b'),
    'FY': Meesho_Merged_1['Financial Year'],
    'Nature': 'B2C',
    'Transaction Type': Meesho_Merged_1['order_status'],
    'Destination': Meesho_Merged_1['end_customer_state_new'],
    'HSN Code': Meesho_Merged_1['HSN'],
    'Qty': Meesho_Merged_1['quantity'],
    'Taxable Value': Meesho_Merged_1['tcs_taxable_amount'],
    'GST Rate': Meesho_Merged_1['gst_rate'] / 100,
    'IGST': 
    Meesho_Merged.apply(
    lambda row: row['tcs_taxable_amount'] * (row['gst_rate'] / 100) if row['end_customer_state_new'] != 'Tamil Nadu' else 0,
    axis=1),
    'CGST': 
    Meesho_Merged.apply(
    lambda row: row['tcs_taxable_amount'] * ((row['gst_rate']  / 100) / 2) if row['end_customer_state_new'] == 'Tamil Nadu' else 0,
    axis=1),
    'SGST': 
    Meesho_Merged.apply(
    lambda row: row['tcs_taxable_amount'] * ((row['gst_rate']  / 100) / 2) if row['end_customer_state_new'] == 'Tamil Nadu' else 0,
    axis=1),
    'Invoice Value': Meesho_Merged_1['meesho_price'],
    'TCS': Meesho_Merged_1['tcs_taxable_amount'] * 0.01
})

Final_Report = pd.concat([Flipkart_Tax_report, B2B_Amazon_Tax_Report, B2C_Amazon_Tax_Report, Meesho_Merged_Forward_Report, Meesho_Merged_1_Reverse_Report], axis=0, ignore_index=True)

Final_Tax_Report = Final_Report[Final_Report[['Invoice Value', 'Invoice Date']].notna().all(axis=1)]

In [35]:
Final_Tax_Report

,Marketplace,Invoice Number,Invoice Date,Party Name,GSTN,Period,FY,Nature,Transaction Type,Destination,HSN Code,Qty,Taxable Value,GST Rate,IGST,CGST,SGST,Invoice Value,TCS
0,FLIPKART,FAHTK32200000001,2021-06-15 00:00:00,NA,NA,Jun,2021-22,B2C,Sale,Uttar Pradesh,35052000,1.0,174.580000,0.18,31.420000,0.0,0.0,206.0,1.750000
1,FLIPKART,FAHTK32200000002,2021-06-20 00:00:00,NA,NA,Jun,2021-22,B2C,Sale,Telangana,35061000,1.0,143.220000,0.18,25.780000,0.0,0.0,169.0,1.430000
2,FLIPKART,FAHTK32200000003,2021-06-20 00:00:00,NA,NA,Jun,2021-22,B2C,Sale,Odisha,35061000,1.0,154.240000,0.18,27.760000,0.0,0.0,182.0,1.540000
3,FLIPKART,RAF0JN2200000001,2021-06-22 00:00:00,NA,NA,Jun,2021-22,B2C,Return,Telangana,35061000,1.0,-143.220000,0.18,-25.780000,0.0,0.0,-169.0,-1.430000
4,FLIPKART,FAHTK32200000004,2021-06-24 00:00:00,NA,NA,Jun,2021-22,B2C,Sale,Gujarat,35061000,1.0,191.530000,0.18,34.470000,0.0,0.0,226.0,1.920000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4320,MEESHO,dszv725C70,2025-02-02 20:34:29,NA,NA,Feb,2024-25,B2C,Cancelled,MAHARASHTRA,854449,1.0,1045.762712,0.18,43.322034,0.0,0.0,1234.0,10.457627
4321,MEESHO,dszv725C71,2025-02-05 21:40:06,NA,NA,Feb,2024-25,B2C,Cancelled,UTTARAKHAND,854449,1.0,500.847458,0.18,43.322034,0.0,0.0,591.0,5.008475
4322,MEESHO,dszv725C73,2025-02-16 19:01:35,NA,NA,Feb,2024-25,B2C,Cancelled,CHHATTISGARH,854449,1.0,544.915254,0.18,43.322034,0.0,0.0,643.0,5.449153
4323,MEESHO,dszv725C69,2025-02-01 14:14:32,NA,NA,Feb,2024-25,B2C,Cancelled,MADHYA PRADESH,854449,1.0,466.101695,0.18,43.322034,0.0,0.0,591.0,4.661017


In [12]:
export_path_1 = 'final_tax_report.json'
export_path_2 = 'meesho_invoice_data.json'
export_path_3 = 'meesho_forward_data.json'
export_path_4 = 'meesho_reverse_data.json'
Final_Tax_Report.to_json(export_path_1, orient='records', date_format='iso', lines=False)
meesho_invoice_data.to_json(export_path_2, orient='records', date_format='iso', lines=False)
meesho_forward_data.to_json(export_path_3, orient='records', date_format='iso', lines=False)
meesho_reverse_data.to_json(export_path_4, orient='records', date_format='iso', lines=False)

client = MongoClient('mongodb://localhost:27017/')
db = client['Ecommerce_DB']
collections = {
    "Final_Report": Final_Tax_Report,
    "Meesho_Invoice_Report": meesho_invoice_data,
    "Meesho_Forward_Report": meesho_forward_data,
    "Meesho_Reverse_Report": meesho_reverse_data
}

def insert_data_to_mongo(collection_name, df, unique_field=None):
    collection = db[collection_name]
    try:
        # Convert DataFrame to dictionary (list of records)
        data_dict = df.to_dict(orient='records')
        
        # If a unique field is specified, create a unique index on that field or fields
        if unique_field:
            if isinstance(unique_field, list):  # If it's a list, create a composite unique index
                # Create a composite index for multiple fields, each field needs to be specified with direction
                index_fields = [(field, 1) for field in unique_field]
                collection.create_index(index_fields, unique=True)
            else:  # If it's a single field, create a unique index on that field
                collection.create_index([(unique_field, 1)], unique=True)
        
        for data in data_dict:
            # If the unique_field is specified, check if the document with the same value already exists
            if unique_field:
                query = {field: data[field] for field in unique_field}
                if collection.find_one(query):
                    print(f"Document with {query} already exists. Skipping insert.")
                    continue
            
            # Insert the data if it doesn't exist
            collection.insert_one(data)
            print(f"Data inserted successfully: {data}")
        
    except Exception as e:
        print(f"Error inserting data into {collection_name}: {e}")

insert_data_to_mongo("Final_Report", Final_Tax_Report, unique_field=["Invoice Number","Transaction Type","FY"])

Document with {'Invoice Number': 'FAHTK32200000001', 'Transaction Type': 'Sale', 'FY': '2021-22'} already exists. Skipping insert.
Document with {'Invoice Number': 'FAHTK32200000002', 'Transaction Type': 'Sale', 'FY': '2021-22'} already exists. Skipping insert.
Document with {'Invoice Number': 'FAHTK32200000003', 'Transaction Type': 'Sale', 'FY': '2021-22'} already exists. Skipping insert.
Document with {'Invoice Number': 'RAF0JN2200000001', 'Transaction Type': 'Return', 'FY': '2021-22'} already exists. Skipping insert.
Document with {'Invoice Number': 'FAHTK32200000004', 'Transaction Type': 'Sale', 'FY': '2021-22'} already exists. Skipping insert.
Document with {'Invoice Number': 'FAHTK32200000005', 'Transaction Type': 'Sale', 'FY': '2021-22'} already exists. Skipping insert.
Document with {'Invoice Number': 'FAHTK32200000006', 'Transaction Type': 'Sale', 'FY': '2021-22'} already exists. Skipping insert.
Document with {'Invoice Number': 'FAHTK32200000007', 'Transaction Type': 'Sale', 

In [13]:
df = pd.DataFrame(list(db['Final_Report'].find()))

In [14]:
df

,_id,Marketplace,Invoice Number,Invoice Date,Party Name,GSTN,Period,FY,Nature,Transaction Type,Destination,HSN Code,Qty,Taxable Value,GST Rate,IGST,CGST,SGST,Invoice Value,TCS
0,67b1c837b039f7ed9cff8815,FLIPKART,FAHTK32200000001,2021-06-15 00:00:00,NA,NA,Jun,2021-22,B2C,Sale,Uttar Pradesh,35052000,1.0,174.580000,0.18,31.420000,0.0,0.0,206.0,1.750000
1,67b1c837b039f7ed9cff8816,FLIPKART,FAHTK32200000002,2021-06-20 00:00:00,NA,NA,Jun,2021-22,B2C,Sale,Telangana,35061000,1.0,143.220000,0.18,25.780000,0.0,0.0,169.0,1.430000
2,67b1c837b039f7ed9cff8817,FLIPKART,FAHTK32200000003,2021-06-20 00:00:00,NA,NA,Jun,2021-22,B2C,Sale,Odisha,35061000,1.0,154.240000,0.18,27.760000,0.0,0.0,182.0,1.540000
3,67b1c837b039f7ed9cff8818,FLIPKART,RAF0JN2200000001,2021-06-22 00:00:00,NA,NA,Jun,2021-22,B2C,Return,Telangana,35061000,1.0,-143.220000,0.18,-25.780000,0.0,0.0,-169.0,-1.430000
4,67b1c837b039f7ed9cff8819,FLIPKART,FAHTK32200000004,2021-06-24 00:00:00,NA,NA,Jun,2021-22,B2C,Sale,Gujarat,35061000,1.0,191.530000,0.18,34.470000,0.0,0.0,226.0,1.920000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4067,67d4459c2df68cfc0d8ac603,MEESHO,dszv725C70,2025-02-02 20:34:29,NA,NA,Feb,2024-25,B2C,Cancelled,MAHARASHTRA,854449,1.0,1045.762712,0.18,43.322034,0.0,0.0,1234.0,10.457627
4068,67d4459c2df68cfc0d8ac604,MEESHO,dszv725C71,2025-02-05 21:40:06,NA,NA,Feb,2024-25,B2C,Cancelled,UTTARAKHAND,854449,1.0,500.847458,0.18,43.322034,0.0,0.0,591.0,5.008475
4069,67d4459c2df68cfc0d8ac605,MEESHO,dszv725C73,2025-02-16 19:01:35,NA,NA,Feb,2024-25,B2C,Cancelled,CHHATTISGARH,854449,1.0,544.915254,0.18,43.322034,0.0,0.0,643.0,5.449153
4070,67d4459c2df68cfc0d8ac606,MEESHO,dszv725C69,2025-02-01 14:14:32,NA,NA,Feb,2024-25,B2C,Cancelled,MADHYA PRADESH,854449,1.0,466.101695,0.18,43.322034,0.0,0.0,591.0,4.661017
